# Imports

In [27]:
from google.cloud import storage
import pandas as pd
import numpy as np
import re
import datetime
from unidecode import unidecode

In [28]:
def get_data(bucket_name:str, imobiliarias:str = ['apolar', 'cilar'],by:str = ['date','date_diff'], dates:list = [], date_diff:int = 2):
    storage_client = storage.Client()
    bucket = storage_client.get_bucket(bucket_name)

    files_on_bucket = [i.name for i in bucket.list_blobs()]
    files = pd.DataFrame(files_on_bucket, columns=['name'])
    files['date'] = pd.to_datetime(files['name'].apply(lambda f: f.split(' - ')[0]))
    files['imobiliaria'] = files['name'].apply(lambda f: f.split(' - ')[-1].replace('.csv',''))

    match by:
        case 'date':
            files = files.loc[files['date'].isin(dates)]
            files = files.loc[files['imobiliaria'].isin(imobiliarias)]
        case 'date_diff':
            dates = files['date'].sort_values(ascending=False).drop_duplicates().reset_index(drop=True)[:date_diff].tolist()
            files = files.loc[files['date'].isin(dates)]
            files = files.loc[files['imobiliaria'].isin(imobiliarias)]
    
    df_full = pd.DataFrame()

    for file_name in files['name'].tolist():
        try:
            df_aux = pd.read_csv(f'gs://{bucket_name}/{file_name}')
            df_full = pd.concat([df_full, df_aux], axis = 0)
        except:
            pass

    df_full = df_full.reset_index(drop=True)

    return df_full

# Carregando Dados

In [29]:
df_full = get_data(
    bucket_name='busca-apartamentos-bucket',
    by='date',
    imobiliarias=['apolar'],
    dates=['2024-03-24']
)

/tmp/ipykernel_835962/864656196.py:12: FutureWarning: The behavior of 'isin' with dtype=datetime64[ns] and castable values (e.g. strings) is deprecated. In a future version, these will not be considered matching by isin. Explicitly cast to the appropriate dtype before calling isin instead.
  files = files.loc[files['date'].isin(dates)]


In [30]:
df = df_full.copy()

In [31]:
df.head()

,site,data_coleta,titulo,link,endereco,valores,atributos,descricao,ficha_tecnica
0,Apolar,2024-03-24,\n Apartamento Residencial para...,https://www.apolar.com.br/alugar/curitiba/agua...,"\n Avenida Silva Jardim, 2733, ...","R$ 20.000,00, , Seguro Incêndio R$ 62,56, Tota...",203 m²,\n Apartamento com\n ...,"Cidade: Curitiba, Área Terreno: 203.09m², Área..."
1,Apolar,2024-03-24,\n Apartamento Residencial para...,https://www.apolar.com.br/alugar/curitiba/cris...,"\n Rua Urbano Lopes, 238, Crist...","R$ 12.000,00, , Condomínio R$ 840,00, Seguro I...","250 m², 1 banheiro, 3 vagas, 3 quartos, 1 suit...",\n Apartamento com\n ...,"Cidade: Curitiba, Área Terreno: 250m², Área To..."
2,Apolar,2024-03-24,\n Apartamento Residencial para...,https://www.apolar.com.br/alugar/curitiba/ecov...,\n Rua Dep. Heitor Alencar Furt...,"R$ 10.500,00, , Condomínio R$ 1.549,67, IPTU R...","330 m², 2 vagas, 4 quartos",\n Apartamento com\n ...,"Cidade: Curitiba, Área Terreno: 330m², Área To..."
3,Apolar,2024-03-24,\n Apartamento Residencial para...,https://www.apolar.com.br/alugar/curitiba/merc...,"\n Avenida Cândido Hartmann, 33...","R$ 7.000,00, , Condomínio R$ 1.690,58, IPTU R$...","163 m², 2 banheiros, 2 vagas, 2 quartos, 1 sui...",\n Apartamento com\n ...,"Cidade: Curitiba, Área Terreno: 163.97m², Área..."
4,Apolar,2024-03-24,\n Apartamento Residencial para...,https://www.apolar.com.br/alugar/curitiba/guab...,\n Avenida Senador Salgado Filh...,"R$ 5.000,00, , Condomínio R$ 635,61, IPTU R$ 2...","107 m², 4 banheiros, 2 vagas, 2 quartos",\n Apartamento com\n ...,"Cidade: Curitiba, Área Terreno: 107.76m², Área..."


# Tratando Dados

## Limpeza

In [32]:
df['titulo'] = df['titulo'].str.strip()
df['endereco'] = df['endereco'].str.strip()
df['descricao'] = df['descricao'].str.strip().str.replace('  ', '').str.replace('\n',' ')

# Criação de Features

In [33]:
def busca_substring(substring, string_list):
    result = np.nan
    for s in string_list:
        if substring in s:
            try:
                result = re.findall(r'\s(\d{1,3}(?:[.,]\d{3})*(?:[.,]\d{2})?)', s)[0]
            except:
                result = s
            break
            
    return result

def separa_valores_imovel(string):

    # Padrao regex para encontrar nome e valor monetário
    padrao = r'(\w+)\sR\$\s(\d{1,3}(?:\.\d{3})*(?:,\d{2})?)'

    # Encontrar todas as correspondências na string
    correspondencias = re.findall(padrao, string)

    # Imprimir os resultados
    list_values = []
    for correspondencia in correspondencias:
        nome, valor = correspondencia
        list_values.append(f'{nome}: {valor}')
    
    return list_values


In [34]:
df

,site,data_coleta,titulo,link,endereco,valores,atributos,descricao,ficha_tecnica
0,Apolar,2024-03-24,Apartamento Residencial para Locação no Água V...,https://www.apolar.com.br/alugar/curitiba/agua...,"Avenida Silva Jardim, 2733, Água Verde - Curitiba","R$ 20.000,00, , Seguro Incêndio R$ 62,56, Tota...",203 m²,Apartamento com no Bairro Água Verde Imóvel c...,"Cidade: Curitiba, Área Terreno: 203.09m², Área..."
1,Apolar,2024-03-24,Apartamento Residencial para Locação no Cristo...,https://www.apolar.com.br/alugar/curitiba/cris...,"Rua Urbano Lopes, 238, Cristo Rei - Curitiba","R$ 12.000,00, , Condomínio R$ 840,00, Seguro I...","250 m², 1 banheiro, 3 vagas, 3 quartos, 1 suit...",Apartamento com 3 quartos no Bairro Cristo Rei...,"Cidade: Curitiba, Área Terreno: 250m², Área To..."
2,Apolar,2024-03-24,Apartamento Residencial para Locação no Ecovil...,https://www.apolar.com.br/alugar/curitiba/ecov...,"Rua Dep. Heitor Alencar Furtado, 3180, Ecovill...","R$ 10.500,00, , Condomínio R$ 1.549,67, IPTU R...","330 m², 2 vagas, 4 quartos",Apartamento com 4 quartos no Bairro Ecoville I...,"Cidade: Curitiba, Área Terreno: 330m², Área To..."
3,Apolar,2024-03-24,Apartamento Residencial para Locação no Mercês...,https://www.apolar.com.br/alugar/curitiba/merc...,"Avenida Cândido Hartmann, 330, Mercês - Curitiba","R$ 7.000,00, , Condomínio R$ 1.690,58, IPTU R$...","163 m², 2 banheiros, 2 vagas, 2 quartos, 1 sui...",Apartamento com 2 quartos no Bairro Mercês Imó...,"Cidade: Curitiba, Área Terreno: 163.97m², Área..."
4,Apolar,2024-03-24,Apartamento Residencial para Locação no Guabir...,https://www.apolar.com.br/alugar/curitiba/guab...,"Avenida Senador Salgado Filho, 1798, Guabirotu...","R$ 5.000,00, , Condomínio R$ 635,61, IPTU R$ 2...","107 m², 4 banheiros, 2 vagas, 2 quartos",Apartamento com 2 quartos no Bairro Guabirotub...,"Cidade: Curitiba, Área Terreno: 107.76m², Área..."
...,...,...,...,...,...,...,...,...,...
370,Apolar,2024-03-24,Apartamento Residencial para Locação no Campo ...,https://www.apolar.com.br/alugar/curitiba/camp...,"Rua Ângelo Tozim, 1220, Campo De Santana - Cur...","R$ 700,00, , Condomínio R$ 380,00, IPTU R$ 20,...","45 m², 1 banheiro, 1 vaga, 2 quartos",Apartamento com 2 quartos no Bairro Campo De S...,"Cidade: Curitiba, Área Terreno: 45.33m², Área ..."
371,Apolar,2024-03-24,Apartamento Residencial para Locação no Campo ...,https://www.apolar.com.br/alugar/curitiba/camp...,"Rua Ângelo Tozim, 1399, Campo De Santana - Cur...","R$ 700,00, , Condomínio R$ 340,00, IPTU R$ 18,...","48 m², 1 banheiro, 1 vaga, 2 quartos, pet",Apartamento com 2 quartos no Bairro Campo De S...,"Cidade: Curitiba, Área Terreno: 48m², Área Tot..."
372,Apolar,2024-03-24,Apartamento Residencial para Locação no Campo ...,https://www.apolar.com.br/alugar/curitiba/camp...,"Rua Ângelo Tozim, 200, Campo De Santana - Curi...","R$ 700,00, , Condomínio R$ 360,00, Seguro Incê...","52 m², 1 banheiro, 1 vaga, 2 quartos, pet",Apartamento com 2 quartos no Bairro Campo De S...,"Cidade: Curitiba, Área Terreno: 52m², Área Tot..."
373,Apolar,2024-03-24,Apartamento Residencial para Locação no Cidade...,https://www.apolar.com.br/alugar/curitiba/cida...,"Rua Professor Algacyr Munhoz Mader, 2411, Cida...","R$ 650,00, , Condomínio R$ 350,88, IPTU R$ 25,...","45 m², 1 banheiro, 1 vaga, 2 quartos",Apartamento com 2 quartos no Bairro Cidade Ind...,"Cidade: Curitiba, Área Terreno: 45m², Área Tot..."


In [35]:
df.loc[24]

site                                                        Apolar
data_coleta                                             2024-03-24
titulo                                                         NaN
link             https://www.apolar.com.br/alugar/curitiba/boa-...
endereco                                                       NaN
valores                                                        NaN
atributos                                                      NaN
descricao                                                      NaN
ficha_tecnica                                                  NaN
Name: 24, dtype: object

In [36]:
# Localidade
df['bairro'] = df['endereco'].apply(lambda x: x if isinstance(x,float) else x.split(', ')[2].split(' - ')[0])
# df['cidade'] = df['endereco'].apply(lambda x: x if isinstance(x,float) else x.split(', ')[2].split(' - ')[1])
try:
    df['cidade'] = df['endereco'].apply(lambda x: x if isinstance(x,float) else x.split(', ')[2].split(' - ')[-1])
except:
    df['cidade'] = np.nan

# Atributos do imóvel
df['area'] = df['atributos'].apply(lambda x: x if isinstance(x,float) else busca_substring('m²', x.split(', ')))
df['banheiros'] = df['atributos'].apply(lambda x: x if isinstance(x,float) else busca_substring('banheiro', x.split(', ')))
# df['vagas_garagem'] = df['atributos'].apply(lambda x: x if isinstance(x,float) else busca_substring('garagem', x.split(', ')))
df['quartos'] = df['atributos'].apply(lambda x: x if isinstance(x,float) else busca_substring('quarto', x.split(', ')))
df['suites'] = df['atributos'].apply(lambda x: x if isinstance(x,float) else busca_substring('suite', x.split(', ')))

df['area'] = df['area'].apply(lambda x: np.nan if isinstance(x, float) else x.split(' ')[0]).astype('float64')
df['banheiros'] = df['banheiros'].apply(lambda x: np.nan if isinstance(x, float) else x.split(' ')[0]).astype('float64')
df['vagas_garagem'] = df['ficha_tecnica'].apply(lambda x: x if isinstance(x,float) else busca_substring('Garagem', x.replace(':','').split(', ')))
df['quartos'] = df['quartos'].apply(lambda x: np.nan if isinstance(x, float) else x.split(' ')[0]).astype('float64')
df['suites'] = df['suites'].apply(lambda x: np.nan if isinstance(x, float) else x.split(' ')[0]).astype('float64')

# Valores
df['aluguel'] = df['valores'].apply(lambda x: x if isinstance(x,float) else x.split(', ')[0].replace('R$ ',''))
df['condominio'] = df['valores'].apply(lambda x: x if isinstance(x,float) else busca_substring('Condomínio',separa_valores_imovel(x)))
df['seguro_incendio'] = df['valores'].apply(lambda x: x if isinstance(x,float) else busca_substring('Incêndio',separa_valores_imovel(x)))
df['iptu'] = df['valores'].apply(lambda x: x if isinstance(x,float) else busca_substring('IPTU',separa_valores_imovel(x)))

df['aluguel'] = df['aluguel'].apply(lambda x: x if isinstance(x,float) else x.replace(',00', '').replace('.','').replace(',','.')).astype('float64')
df['condominio'] = df['condominio'].apply(lambda x: x if isinstance(x,float) else x.replace(',00', '').replace('.','').replace(',','.')).astype('float64')
df['seguro_incendio'] = df['seguro_incendio'].apply(lambda x: x if isinstance(x,float) else x.replace(',00', '').replace('.','').replace(',','.')).astype('float64')
df['iptu'] = df['iptu'].apply(lambda x: x if isinstance(x,float) else x.replace(',00', '').replace('.','').replace(',','.')).astype('float64')

# Detalhes do imóvel/condomínio
df['mobiliado'] = df['descricao'].apply(lambda x: np.nan if isinstance(x,float) else 'Sim' if 'mobiliado' in unidecode(x.lower()) else 'Não')
df['piscina'] = df['descricao'].apply(lambda x: np.nan if isinstance(x,float) else 'Sim' if 'piscina' in unidecode(x.lower()) else 'Não')
df['academia'] = df['descricao'].apply(lambda x: np.nan if isinstance(x,float) else 'Sim' if 'academia' in unidecode(x.lower()) else 'Não')
df['sacada'] = df['descricao'].apply(lambda x: np.nan if isinstance(x,float) else 'Sim' if 'sacada' in unidecode(x.lower()) else 'Não')
df['churrasqueira'] = df['descricao'].apply(lambda x: np.nan if isinstance(x,float) else 'Sim' if 'churrasqueira' in unidecode(x.lower()) else 'Não')
df['salao_de_festas'] = df['descricao'].apply(lambda x: np.nan if isinstance(x,float) else 'Sim' if 'salao de festas' in unidecode(x.lower()) else 'Não')

####### TRATAMENTO FINAL DE DADOS ######

## Transformando dtypes
df['data_coleta'] = pd.to_datetime(df['data_coleta'])
df['cidade'] = df['cidade'].apply(lambda x: x if isinstance(x,float) else unidecode(x.capitalize())).astype('category')
df['bairro'] = df['bairro'].apply(lambda x: x if isinstance(x,float) else unidecode(x.capitalize())).astype('category')
df[['area','quartos','suites','banheiros','vagas_garagem']] = df[['area','quartos','suites','banheiros','vagas_garagem']].astype('float64')
df[['aluguel','condominio','seguro_incendio','iptu']] = df[['aluguel','condominio','seguro_incendio','iptu']].astype('float64')


In [37]:
df.isna().sum()

site                 0
data_coleta          0
titulo               7
link                 0
endereco             7
valores              7
atributos            7
descricao            7
ficha_tecnica        7
bairro               7
cidade               7
area                 7
banheiros          130
quartos              9
suites             324
vagas_garagem       80
aluguel              7
condominio          35
seguro_incendio      7
iptu                93
mobiliado            7
piscina              7
academia             7
sacada               7
churrasqueira        7
salao_de_festas      7
dtype: int64

# Save File

In [38]:
columns_selected = [
    'site',
    'titulo',
    'link',
    'data_coleta',
    'endereco',
    'atributos',
    'descricao',
    'bairro',
    'cidade',
    'aluguel',
    'condominio',
    'seguro_incendio',
    'iptu',
    'area',
    'quartos',
    'suites',
    'banheiros',
    'vagas_garagem',
    'mobiliado',
    'piscina',
    'academia',
    'sacada',
    'churrasqueira',
    'salao_de_festas'
 ]

df = df.loc[df['cidade'] == 'Curitiba']

In [39]:
df

,site,data_coleta,titulo,link,endereco,valores,atributos,descricao,ficha_tecnica,bairro,...,aluguel,condominio,seguro_incendio,iptu,mobiliado,piscina,academia,sacada,churrasqueira,salao_de_festas
0,Apolar,2024-03-24,Apartamento Residencial para Locação no Água V...,https://www.apolar.com.br/alugar/curitiba/agua...,"Avenida Silva Jardim, 2733, Água Verde - Curitiba","R$ 20.000,00, , Seguro Incêndio R$ 62,56, Tota...",203 m²,Apartamento com no Bairro Água Verde Imóvel c...,"Cidade: Curitiba, Área Terreno: 203.09m², Área...",Agua verde,...,20000.0,NaN,62.56,NaN,Não,Não,Não,Não,Não,Não
1,Apolar,2024-03-24,Apartamento Residencial para Locação no Cristo...,https://www.apolar.com.br/alugar/curitiba/cris...,"Rua Urbano Lopes, 238, Cristo Rei - Curitiba","R$ 12.000,00, , Condomínio R$ 840,00, Seguro I...","250 m², 1 banheiro, 3 vagas, 3 quartos, 1 suit...",Apartamento com 3 quartos no Bairro Cristo Rei...,"Cidade: Curitiba, Área Terreno: 250m², Área To...",Cristo rei,...,12000.0,840.00,71.93,NaN,Sim,Não,Sim,Sim,Sim,Sim
2,Apolar,2024-03-24,Apartamento Residencial para Locação no Ecovil...,https://www.apolar.com.br/alugar/curitiba/ecov...,"Rua Dep. Heitor Alencar Furtado, 3180, Ecovill...","R$ 10.500,00, , Condomínio R$ 1.549,67, IPTU R...","330 m², 2 vagas, 4 quartos",Apartamento com 4 quartos no Bairro Ecoville I...,"Cidade: Curitiba, Área Terreno: 330m², Área To...",Ecoville,...,10500.0,1549.67,87.58,21.37,Não,Não,Não,Não,Não,Não
3,Apolar,2024-03-24,Apartamento Residencial para Locação no Mercês...,https://www.apolar.com.br/alugar/curitiba/merc...,"Avenida Cândido Hartmann, 330, Mercês - Curitiba","R$ 7.000,00, , Condomínio R$ 1.690,58, IPTU R$...","163 m², 2 banheiros, 2 vagas, 2 quartos, 1 sui...",Apartamento com 2 quartos no Bairro Mercês Imó...,"Cidade: Curitiba, Área Terreno: 163.97m², Área...",Merces,...,7000.0,1690.58,53.19,341.74,Não,Não,Sim,Não,Sim,Não
4,Apolar,2024-03-24,Apartamento Residencial para Locação no Guabir...,https://www.apolar.com.br/alugar/curitiba/guab...,"Avenida Senador Salgado Filho, 1798, Guabirotu...","R$ 5.000,00, , Condomínio R$ 635,61, IPTU R$ 2...","107 m², 4 banheiros, 2 vagas, 2 quartos",Apartamento com 2 quartos no Bairro Guabirotub...,"Cidade: Curitiba, Área Terreno: 107.76m², Área...",Guabirotuba,...,5000.0,635.61,39.14,230.98,Não,Não,Não,Sim,Sim,Não
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
370,Apolar,2024-03-24,Apartamento Residencial para Locação no Campo ...,https://www.apolar.com.br/alugar/curitiba/camp...,"Rua Ângelo Tozim, 1220, Campo De Santana - Cur...","R$ 700,00, , Condomínio R$ 380,00, IPTU R$ 20,...","45 m², 1 banheiro, 1 vaga, 2 quartos",Apartamento com 2 quartos no Bairro Campo De S...,"Cidade: Curitiba, Área Terreno: 45.33m², Área ...",Campo de santana,...,700.0,380.00,26.26,20.58,Não,Não,Não,Sim,Não,Não
371,Apolar,2024-03-24,Apartamento Residencial para Locação no Campo ...,https://www.apolar.com.br/alugar/curitiba/camp...,"Rua Ângelo Tozim, 1399, Campo De Santana - Cur...","R$ 700,00, , Condomínio R$ 340,00, IPTU R$ 18,...","48 m², 1 banheiro, 1 vaga, 2 quartos, pet",Apartamento com 2 quartos no Bairro Campo De S...,"Cidade: Curitiba, Área Terreno: 48m², Área Tot...",Campo de santana,...,700.0,340.00,26.26,18.53,Não,Não,Sim,Sim,Não,Sim
372,Apolar,2024-03-24,Apartamento Residencial para Locação no Campo ...,https://www.apolar.com.br/alugar/curitiba/camp...,"Rua Ângelo Tozim, 200, Campo De Santana - Curi...","R$ 700,00, , Condomínio R$ 360,00, Seguro Incê...","52 m², 1 banheiro, 1 vaga, 2 quartos, pet",Apartamento com 2 quartos no Bairro Campo De S...,"Cidade: Curitiba, Área Terreno: 52m², Área Tot...",Campo de santana,...,700.0,360.00,26.26,NaN,Não,Não,Sim,Não,Não,Sim
373,Apolar,2024-03-24,Apartamento Residencial para Locação no Cidade...,https://www.apolar.com.br/alugar/curitiba/cida...,"Rua Professor Algacyr Munhoz Mader, 2411, Cida...","R$ 650,00, , Condomínio R$ 350,88, IPTU R$ 25,...","45 m², 1 banheiro, 1 vaga, 2 quartos",Apartam

In [40]:
# for date in files['date'].astype(str).tolist():
# df_aux = df.loc[df['data_coleta'] == str(date)]

BUCKET_NAME = 'busca-apartamentos-trusted'
FILE_NAME = f'2024-03-24 - apartamentos - apolar.csv'
TEMP_FILE = 'local.csv'
RAW_PATH = '/tmp/{}'.format(TEMP_FILE)

storage_client = storage.Client()
bucket = storage_client.get_bucket(BUCKET_NAME)

df[columns_selected].to_csv(RAW_PATH, index=False)
blob = bucket.blob(FILE_NAME)
blob.upload_from_filename(RAW_PATH)

In [69]:
df

,site,data_coleta,titulo,link,endereco,valores,atributos,descricao,bairro,cidade,...,aluguel,condominio,seguro_incendio,iptu,mobiliado,piscina,academia,sacada,churrasqueira,salao_de_festas
0,Apolar,2024-02-21,Apartamento Residencial para Locação no Água V...,https://www.apolar.com.br/alugar/curitiba/agua...,"Avenida Silva Jardim, 2733, Água Verde - Curitiba","Aluguel R$ 20.000,00 Seguro Incêndio R$ 62,56T...",203 m²,Apartamento com no Bairro Água Verde Imóvel c...,Agua verde,Curitiba,...,20000.0,NaN,62.56,NaN,Não,Não,Não,Não,Não,Não
1,Apolar,2024-02-21,Apartamento Residencial para Locação no Cristo...,https://www.apolar.com.br/alugar/curitiba/cris...,"Rua Urbano Lopes, 238, Cristo Rei - Curitiba","Aluguel R$ 12.000,00 Condomínio R$ 840,00 Segu...","250 m², 1 banheiro, 3 vagas, 3 quartos, 1 suit...",Apartamento com 3 quartos no Bairro Cristo Rei...,Cristo rei,Curitiba,...,12000.0,840.00,71.93,NaN,Sim,Não,Sim,Sim,Sim,Sim
2,Apolar,2024-02-21,Apartamento Residencial para Locação no Ecovil...,https://www.apolar.com.br/alugar/curitiba/ecov...,"Rua Dep. Heitor Alencar Furtado, 3180, Ecovill...","Aluguel R$ 10.500,00 Condomínio R$ 1.549,67 Se...","330 m², 2 vagas, 4 quartos",Apartamento com 4 quartos no Bairro Ecoville I...,Ecoville,Curitiba,...,10500.0,1549.67,87.58,NaN,Não,Não,Não,Não,Não,Não
3,Apolar,2024-02-21,Apartamento Residencial para Locação no Mercês...,https://www.apolar.com.br/alugar/curitiba/merc...,"Avenida Cândido Hartmann, 330, Mercês - Curitiba","Aluguel R$ 7.500,00 Condomínio R$ 1.690,58 Seg...","163 m², 2 banheiros, 2 vagas, 2 quartos, 1 sui...",Apartamento com 2 quartos no Bairro Mercês Imó...,Merces,Curitiba,...,7500.0,1690.58,53.19,NaN,Não,Não,Sim,Não,Sim,Não
4,Apolar,2024-02-21,Apartamento Residencial para Locação no Guabir...,https://www.apolar.com.br/alugar/curitiba/guab...,"Avenida Senador Salgado Filho, 1798, Guabirotu...","Aluguel R$ 5.000,00 Condomínio R$ 635,61 Segur...","107 m², 4 banheiros, 2 vagas, 2 quartos",Apartamento com 2 quartos no Bairro Guabirotub...,Guabirotuba,Curitiba,...,5000.0,635.61,39.14,NaN,Não,Não,Não,Sim,Sim,Não
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
647,Apolar,2024-03-02,Apartamento Residencial para Locação no Jardim...,https://www.apolar.com.br/alugar/campo-largo/j...,"Rua Maria Joaquina Vaz, 721, Jardim Itaqui - C...","Aluguel R$ 600,00, , Condomínio R$ 320,00, Seg...","61 m², 1 banheiro, 1 vaga, 2 quartos, pet",Apartamento com 2 quartos no Bairro Jardim Ita...,Jardim itaqui,Campo largo,...,600.0,320.00,28.60,NaN,Não,Sim,Sim,Sim,Não,Sim
648,Apolar,2024-03-02,Apartamento Residencial para Locação no Uvaran...,https://www.apolar.com.br/alugar/ponta-grossa/...,"Avenida General Carlos Cavalcanti, 5685, Uvara...","Aluguel R$ 600,00, , Condomínio R$ 350,00, IPT...","57 m², 1 banheiro, 1 vaga, 2 quartos, pet",Apartamento com 2 quartos no Bairro Uvaranas I...,Uvaranas,Ponta grossa,...,600.0,350.00,27.43,83.9,Não,Não,Não,Não,Não,Não
649,Apolar,2024-03-02,Apartamento Residencial para Locação no Jardim...,https://www.apolar.com.br/alugar/campo-largo/j...,"Rua Maria Joaquina Vaz, 721, Jardim Itaqui - C...","Aluguel R$ 600,00, , Condomínio R$ 265,00, Seg...","46 m², 2 quartos",Apartamento com 2 quartos no Bairro Jardim Ita...,Jardim itaqui,Campo largo,...,600.0,265.00,26.26,NaN,Não,Sim,Não,Não,Não,Sim
650,Apolar,2024-03-02,ref.,https://www.apolar.com.br/alugar/campo-largo/j...,", , -","Aluguel R$ 0,00,",NaN,NaN,-,-,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [45]:
df_full

,site,data_coleta,titulo,link,endereco,valores,atributos,descricao
0,Apolar,2024-03-09,\n Apartamento Residencial para...,https://www.apolar.com.br/alugar/curitiba/agua...,"\n Avenida Silva Jardim, 2733, ...","R$ 20.000,00, , Seguro Incêndio R$ 62,56, Tota...",203 m²,\n Apartamento com\n ...
1,Apolar,2024-03-09,\n Apartamento Residencial para...,https://www.apolar.com.br/alugar/curitiba/cris...,"\n Rua Urbano Lopes, 238, Crist...","R$ 12.000,00, , Condomínio R$ 840,00, Seguro I...","250 m², 1 banheiro, 3 vagas, 3 quartos, 1 suit...",\n Apartamento com\n ...
2,Apolar,2024-03-09,\n Apartamento Residencial para...,https://www.apolar.com.br/alugar/curitiba/ecov...,\n Rua Dep. Heitor Alencar Furt...,"R$ 10.500,00, , Condomínio R$ 1.549,67, IPTU R...","330 m², 2 vagas, 4 quartos",\n Apartamento com\n ...
3,Apolar,2024-03-09,\n Apartamento Residencial para...,https://www.apolar.com.br/alugar/curitiba/merc...,"\n Avenida Cândido Hartmann, 33...","R$ 7.000,00, , Condomínio R$ 1.690,58, IPTU R$...","163 m², 2 banheiros, 2 vagas, 2 quartos, 1 sui...",\n Apartamento com\n ...
4,Apolar,2024-03-09,\n Apartamento Residencial para...,https://www.apolar.com.br/alugar/curitiba/guab...,\n Avenida Senador Salgado Filh...,"R$ 5.000,00, , Condomínio R$ 635,61, IPTU R$ 2...","107 m², 4 banheiros, 2 vagas, 2 quartos",\n Apartamento com\n ...
...,...,...,...,...,...,...,...,...
368,Apolar,2024-03-09,\n Apartamento Residencial para...,https://www.apolar.com.br/alugar/curitiba/barr...,"\n Rua Maria De Lourdes Kudri, ...","R$ 700,00, , IPTU R$ 20,58, Seguro Incêndio R$...","41 m², 2 quartos",\n Apartamento com\n ...
369,Apolar,2024-03-09,\n Apartamento Residencial para...,https://www.apolar.com.br/alugar/curitiba/camp...,"\n Rua Ângelo Tozim, 200, Campo...","R$ 700,00, , Condomínio R$ 360,00, Seguro Incê...","52 m², 1 banheiro, 1 vaga, 2 quartos, pet",\n Apartamento com\n ...
370,Apolar,2024-03-09,\n Apartamento Residencial para...,https://www.apolar.com.br/alugar/curitiba/alto...,"\n Rua Capitão Amin Mosse, 101,...","R$ 700,00, , Seguro Incêndio R$ 26,26, Total R...","50 m², 1 banheiro, 1 quarto, pet",\n Apartamento com\n ...
371,Apolar,2024-03-09,\n Apartamento Residencial para...,https://www.apolar.com.br/alugar/curitiba/cida...,\n Rua Professor Algacyr Munhoz...,"R$ 650,00, , Condomínio R$ 350,88, IPTU R$ 25,...","45 m², 1 banheiro, 1 vaga, 2 quartos",\n Apartamento com\n ...


In [44]:
BUCKET_NAME = 'busca-apartamentos-bucket'
FILE_NAME = f'2024-02-18 - apartamentos - apolar.csv'
TEMP_FILE = 'local.csv'
RAW_PATH = '/tmp/{}'.format(TEMP_FILE)

storage_client = storage.Client()
bucket = storage_client.get_bucket(BUCKET_NAME)

df_full.to_csv(RAW_PATH, index=False)
blob = bucket.blob(FILE_NAME)
blob.upload_from_filename(RAW_PATH)